In [172]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np 

import pandas as pd
import sklearn as sklearn
import scipy.stats as stats
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import StratifiedKFold, ParameterGrid
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier

from utils.randomized_search import logUnifD
from utils.base_set import X_train, y_train, seed
from utils.estimators import DecisionTree

### Decision Tree

In [173]:
model1 = DecisionTree()

In [174]:
k = 5
n = 30
N = X_train.shape[1]
param = {
    'criterion': ["gini","entropy","log_loss"],
    'max_depth': stats.randint(1, N)
}

In [175]:
classifier_Dtree = RandomizedSearchCV(estimator = model1.estimator, 
                                param_distributions = param,
                                cv = k,
                                n_iter = n,
                                scoring = "roc_auc"
                               )

In [176]:
classifier_Dtree.fit(X_train,y_train)

/usr/lib/python3/dist-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/lib/python3/dist-packages/sklearn/tree/_classes.py", line 890, in fit
    super().fit(
  File "/usr/lib/python3/dist-packages/sklearn/tree/_classes.py", line 333, in fit
    criterion = CRITERIA_CLF[self.criterion](self.n_outputs_,
KeyError: 'log_loss'

  warnings.warn("Estimator fit failed. The score on this train-test"
/usr/lib/python3/dist-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/lib/python3/d

/usr/lib/python3/dist-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/lib/python3/dist-packages/sklearn/tree/_classes.py", line 890, in fit
    super().fit(
  File "/usr/lib/python3/dist-packages/sklearn/tree/_classes.py", line 333, in fit
    criterion = CRITERIA_CLF[self.criterion](self.n_outputs_,
KeyError: 'log_loss'

  warnings.warn("Estimator fit failed. The score on this train-test"
/usr/lib/python3/dist-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/lib/python3/d

RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_iter=30,
                   param_distributions={'criterion': ['gini', 'entropy',
                                                      'log_loss'],
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x76e8b07b4bb0>},
                   scoring='roc_auc')

In [177]:
columns_to_keep = ['param_max_depth','param_criterion','mean_test_score','rank_test_score']
pd.DataFrame(classifier_Dtree.cv_results_)[columns_to_keep]

,param_max_depth,param_criterion,mean_test_score,rank_test_score
0,75,gini,0.563735,14
1,8,log_loss,NaN,26
2,81,entropy,0.574399,10
3,177,gini,0.549572,18
4,199,gini,0.551630,17
5,115,entropy,0.594665,2
6,102,gini,0.570907,12
7,28,log_loss,NaN,23
8,5,gini,0.542377,19
9,173,gini,0.580584,7


### KNN 

In [178]:
model2 = KNeighborsClassifier()
K = len(X_train)
n = 100
param = {
    'n_neighbors': logUnifD(10,K/2)
}

In [179]:
classifier_KNN = RandomizedSearchCV(estimator = model2, 
                                param_distributions = param,
                                cv = k,
                                n_iter = n,
                                scoring = "roc_auc", 
                                random_state = seed)

In [180]:
classifier_KNN.fit(X_train,y_train)

RandomizedSearchCV(cv=5, estimator=KNeighborsClassifier(), n_iter=100,
                   param_distributions={'n_neighbors': <utils.randomized_search.logUnifD object at 0x76e8b07cf700>},
                   random_state=8241, scoring='roc_auc')

In [181]:
columns_to_keep = ['param_n_neighbors','mean_test_score','rank_test_score']
pd.DataFrame(classifier_KNN.cv_results_)[columns_to_keep]

,param_n_neighbors,mean_test_score,rank_test_score
0,14,0.813417,3
1,166,0.769625,83
2,153,0.772029,82
3,34,0.805245,35
4,93,0.784418,76
...,...,...,...
95,38,0.802264,39
96,15,0.810649,18
97,159,0.769436,84
98,173,0.767385,86


### SVM

In [182]:
model3 = svm.SVC()
n = 100
param = {
    'C': stats.expon(scale=100), 
    'gamma': stats.expon(scale=.1),
    'kernel': ["rbf","linear","poly","sigmoid"]
        }

In [183]:
classifier_SVM = RandomizedSearchCV(estimator = model3, 
                                param_distributions = param,
                                cv = k,
                                n_iter = n,
                                scoring = "roc_auc", 
                                random_state = seed
                               )

In [184]:
classifier_SVM.fit(X_train,y_train)

RandomizedSearchCV(cv=5, estimator=SVC(), n_iter=100,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x76e8b09852d0>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x76e8b09859f0>,
                                        'kernel': ['rbf', 'linear', 'poly',
                                                   'sigmoid']},
                   random_state=8241, scoring='roc_auc')

In [185]:
columns_to_keep = ['param_C','param_gamma','param_kernel','mean_test_score','rank_test_score']
pd.DataFrame(classifier_SVM.cv_results_)[columns_to_keep]

,param_C,param_gamma,param_kernel,mean_test_score,rank_test_score
0,12.587597,0.234105,sigmoid,0.616457,64
1,239.733755,0.041652,poly,0.791603,1
2,143.974508,0.081293,sigmoid,0.612601,68
3,15.163046,0.069363,sigmoid,0.614574,65
4,148.743757,0.468783,rbf,0.500000,82
...,...,...,...,...,...
95,132.631485,0.075499,poly,0.791603,1
96,61.801592,0.09716,poly,0.791603,1
97,16.13484,0.00918,sigmoid,0.633043,61
98,60.395524,0.117131,poly,0.791603,1


### LDA 

In [166]:
model4 = LinearDiscriminantAnalysis()
param = {
    'solver': ["svd","lsqr"], 
    'n_components' : [0,1]
}

In [167]:
classifier_LDA = RandomizedSearchCV(estimator = model4,
                                   param_distributions=param,
                                   cv = k, 
                                   scoring= "roc_auc",
                                   random_state=seed)

In [168]:
classifier_LDA.fit(X_train,y_train)

/usr/lib/python3/dist-packages/sklearn/model_selection/_search.py:278: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV(cv=5, estimator=LinearDiscriminantAnalysis(),
                   param_distributions={'n_components': [0, 1],
                                        'solver': ['svd', 'lsqr']},
                   random_state=8241, scoring='roc_auc')

In [169]:
columns_to_keep = ['param_solver','param_n_components','mean_test_score','rank_test_score']
pd.DataFrame(classifier_LDA.cv_results_)[columns_to_keep]

,param_solver,param_n_components,mean_test_score,rank_test_score
0,svd,0,0.716569,1
1,lsqr,0,0.716569,1
2,svd,1,0.716569,1
3,lsqr,1,0.716569,1


### Naive Bayes 